# Assignment 2



Calculate the forces in each member and reaction forces at the supports. Assume AE is constant. Green arrows show far/end nodes. **Show all work.**

<img src="https://storage.googleapis.com/nm-static/computing_maloof2_20201001.png" alt="nyu_comp_f20">


In [456]:
#Import libraries
import numpy as np



In [457]:
#Storing Nodes & Members: 
##(x,y) format for all values in nodes
##Force is list if at support. 'None' when support allows force in that direction
##Displ is tuple if pin or fixed

node_1 = {
    'name': 'node 1',
    'coord': (4, 6),
    'dof': (1, 2),
    'force': (-500, 0),
    'displ': [None, None]
}

node_2 = {
    'name': 'node 2',
    'coord': (0, 0),
    'dof': (3, 4),
    'force': [None, None],
    'displ': (0, 0)
}

node_3 = {
    'name': 'node 3',
    'coord': (4, 0),
    'dof': (5, 6),
    'force': [None, None],
    'displ': (0, 0)
}

node_4 = {
    'name': 'node 4',
    'coord': (7, 0),
    'dof': (7, 8),
    'force': [None, None],
    'displ': (0, 0)
}


nodes = [
    node_1,
    node_2,
    node_3,
    node_4,
]

#When assignmening members make sure closer node is first

members = [
    (node_1, node_2),
    (node_1, node_3),
    (node_1, node_4)
]

In [458]:
def calc_length(xy1: tuple, xy2: tuple) -> float:
    """ Given the coordinates of two points, it will calculate the
    distance between the two points.
    """
    return np.linalg.norm(np.array(xy1) - np.array(xy2))


def dir_cos(xy1, xy2):
    """ Returns the direction cosine for a member given point coordinates.
    """
    l = calc_length(xy1, xy2)
    return (xy2[0] - xy1[0])/l, (xy2[1] - xy1[1])/l




In [459]:
def element_stiff_matrix(xy1, xy2, other_dofs=None):
    """ Returns the stiffness matrix for a member, given the coordinates of 
    its near and far end nodes. Can optionally provide a tuple that includes 
    other degrees of freedom in order to compile a matrix corresponding 
    to the entire system.
    """
    length = calc_length(xy1, xy2)
    lx, ly = dir_cos(xy1, xy2)
    k = np.array(
        [
        [lx**2, lx*ly, -lx**2, -lx*ly],
        [lx*ly, ly**2, -lx*ly, -ly**2],
        [-lx**2, -lx*ly, lx**2, lx*ly],
        [-lx*ly, -ly**2, lx*ly, ly**2]
        ]
    ) / length
    
    if other_dofs:
        for i in other_dofs:
            k = np.insert(k, i-1, 0, axis=0)
            k = np.insert(k, i-1, 0, axis=1)
    
    return k

def global_stiff_matrix(nodes, members):
    """ It returns the global stiffness matrix given a list of members
    and their nodes, formated as:

    members = [
        (node_a, node_b),
        (node_a, node_d),
        (node_b, node_d),
        (node_b, node_e),
        (node_b, node_c),
        (node_c, node_e),
        (node_e, node_d)
    ]

    and each node as:

    node_a = {
        'coord': (0, 6),
        'dof': (1, 2),
        'force': (150, 0),
        'displ': [None, None]
    }
    """

    k = np.zeros((2*len(nodes), 2*len(nodes)))

    for member in members:
        n1, n2 = member
        all_dof = [i for i in range(1, 2*len(nodes)+1)]
        member_dof = n1['dof'] + n2['dof']
        other_dofs = [i for i in all_dof if i not in member_dof]
        k += element_stiff_matrix(
            n1['coord'], n2['coord'], other_dofs=other_dofs)

    return k



In [460]:
def node_dq(nodes, k):
    """ Given nodes and global stiffness matrix, Returns displacement and forces"""
 # Preparing the displacement arrays
    D = []
    for node in nodes:
        D += list(node['displ'])
    D = np.reshape(np.array(D), (2*len(nodes), 1)).astype(float)
    d_dof = [False if np.isnan(i[0]) else True for i in D]

    # Preparing the force arrays
    Q = []
    for node in nodes:
        Q += list(node['force'])
    Q = np.reshape(np.array(Q), (2*len(nodes), 1)).astype(float)
    q_dof = [False if np.isnan(i[0]) else True for i in Q]

    # Solving for displacements
    dd = np.linalg.inv(k[np.ix_(q_dof, q_dof)]).dot(Q[q_dof])

    # Solving for forces
    qq = k[np.ix_(d_dof, q_dof)].dot(dd)
    # Assume that we always fill in 0 for known q
    for i, b in enumerate(d_dof):
        if b is False:
            qq = np.insert(qq, i, 0, axis=0)

    # Assume that we always fill in 0 for known d
    for i, b in enumerate(q_dof):
        if b is False:
            dd = np.insert(dd, i, 0, axis=0)
    
    return dd, qq



In [461]:
k = global_stiff_matrix(nodes, members)

dd, kk = node_dq(nodes, k)

j = 0

for i in range(len(nodes)):
    print(nodes[i]['name']) 
    print('displacement: \n',  'x: ', dd[j], '\n', 'y: ', dd[j+1])
    j = j + 2

j = 0

for i in range(len(nodes)):
    print(nodes[i]['name']) 
    print('force: \n',  'x: ', kk[j], '\n', 'y: ', kk[j+1])
    j = j + 2

node 1
displacement: 
 x:  [-6902.89657766] 
 y:  [79.07968911]
node 2
displacement: 
 x:  [0.] 
 y:  [0.]
node 3
displacement: 
 x:  [0.] 
 y:  [0.]
node 4
displacement: 
 x:  [0.] 
 y:  [0.]
node 1
force: 
 x:  [0.] 
 y:  [0.]
node 2
force: 
 x:  [289.4799852] 
 y:  [434.21997779]
node 3
force: 
 x:  [0.] 
 y:  [-13.17994819]
node 4
force: 
 x:  [210.5200148] 
 y:  [-421.04002961]
